In [1]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import time
inicio = time.time()

In [2]:
def extract_data():
    raiz = Path.cwd() #capturo diretório atual
    pasta_dados = Path.joinpath(raiz,"dados") #defino onde estão os problemas
    prob_por_arquivo = 200 #informando quantos problemas por arquivo eu tenho
    conjuntos = [10,20,50,100,200,500,1000]

    for conjunto in conjuntos:  #circulo nos arquivos
        dados[conjunto] = {}
        arquivo = f"sch"+str(conjunto)+'.txt'
        print("Iniciando problema {}".format(arquivo))
        caminho_problema = Path.joinpath(pasta_dados,arquivo)
        df = pd.read_fwf(caminho_problema) #importo no df
        df.columns=["n","p","a","b"]
        df.drop(columns=["n"], inplace=True) #jogo fora columa inútil
        n=df.iloc[0,0] #guardo o tamanho de itens do problema
        print("n = ", n)  
        df.dropna(inplace=True) #jogo fora linhas inúteis
        for problema_numero in range(prob_por_arquivo): #vou circular nos problemas dentro do arquivo. Cada iteração deste for é um problema.
            prob = df[n*problema_numero:n*(problema_numero+1)]
            dados[conjunto][problema_numero+1] = {
                'pi': prob["p"].to_numpy(),
                'ai': prob["a"].to_numpy(),
                'bi': prob["b"].to_numpy()
            }
            ## nesse ponto tenho o parâmetros do problema atual guardadas nas variáveis pi, ai e bi, prontos para serem utilizados na modelagem.
    return dados

In [3]:
dados = {}
objetivos = {}
variaveis = {}
tempos = {}
dados = extract_data()
lista_hs = [0.8, 0.6, 0.4, 0.2]

h=0.8
conjunto = 1000
problema = 2
z_corte = 2

pi = []
ai = []
bi = []

pi = np.array(dados[conjunto][problema]['pi'])
ai = np.array(dados[conjunto][problema]['ai'])
bi = np.array(dados[conjunto][problema]['bi'])

d=sum(pi)*h

i_itens = len(pi)
i_index=list(range(i_itens))
i_np = np.array(i_index)

Iniciando problema sch10.txt
n =  10
Iniciando problema sch20.txt
n =  20
Iniciando problema sch50.txt
n =  50
Iniciando problema sch100.txt
n =  100
Iniciando problema sch200.txt
n =  200
Iniciando problema sch500.txt
n =  500
Iniciando problema sch1000.txt
n =  1000


In [4]:


sa = tuple([True,False]*int(i_itens*0.5)) #solucao atual

In [5]:
sa
 

(True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 Fa

In [6]:
def transforma(solucao_partida):
    set_E = np.nonzero(solucao_partida == True)[0] #[0] para retornar o np array e não o tuple.
    set_T = np.nonzero(solucao_partida == False)[0]
    return set_E,set_T

def busca_total(solucao_partida): 
    
    objetivos = np.array([])
    sol_testada = np.array([],dtype=bool)
    for cada_serv in range(len(solucao_partida)):
        sol_em_teste = np.array(solucao_partida)
        sol_em_teste[cada_serv] = not(solucao_partida[cada_serv]) #inverte o serviço de bolso.
        set_E = np.nonzero(sol_em_teste == True)[0] #[0] para retornar o np array e não o tuple.
        set_T = np.nonzero(sol_em_teste == False)[0]
        objetivos = np.append(objetivos,calcula_objetivo(set_E,set_T))
        sol_testada = np.append(sol_testada,sol_em_teste)
    n = len(objetivos)
    sol_testada = np.reshape(sol_testada,[n,n])
    return objetivos,sol_testada


def calcula_objetivo(set_E,set_T):
    if np.sum(pi[set_E])>d:

        return 1e15 #retorna valor imenso.

    ai_pi = ai[set_E]/pi[set_E] #apenas do set_E
    bi_pi = bi[set_T]/pi[set_T] #apenas do set_T
    set_E_pi = pi[set_E]
    set_T_pi = pi[set_T]
    ai_pi_decr = np.flip(np.lexsort((set_E_pi,ai_pi))) #ordem de ai_pi e depois por pi.
    bi_pi_decr = np.flip(np.lexsort((set_T_pi,bi_pi)))
    set_E = set_E[ai_pi_decr]
    set_T = set_T[bi_pi_decr]
    
    
    sum_pi = 0
    objetivo = 0
    for tarefa in set_E:
        objetivo += sum_pi * ai[tarefa]
        sum_pi += pi[tarefa]
    
    sum_pi = 0
    for tarefa in set_T:
        objetivo += (sum_pi+pi[tarefa])*bi[tarefa]
        sum_pi += pi[tarefa]


    
    return objetivo#,True
                           
 

In [7]:
%time objs,sols = busca_total(sa)

Wall time: 2.93 s


In [8]:
np.average(objs)

9072613.786

In [9]:
objs

array([9078001., 9058606., 9079025., 9035921., 9063804., 9067396.,
       9062212., 9075192., 9081694., 9061613., 9077355., 9067039.,
       9081366., 9067953., 9073567., 9062259., 9078413., 9071167.,
       9075543., 9079396., 9082705., 9051333., 9071479., 9081185.,
       9071461., 9060575., 9062088., 9064804., 9077124., 9043040.,
       9095195., 9061879., 9077753., 9092153., 9082052., 9060301.,
       9080745., 9067643., 9076653., 9076190., 9071005., 9071421.,
       9088480., 9060429., 9081219., 9094579., 9094610., 9075807.,
       9081986., 9060307., 9087150., 9059695., 9074751., 9062763.,
       9054317., 9078887., 9074159., 9056058., 9088833., 9084377.,
       9072388., 9063751., 9088660., 9072624., 9089719., 9063977.,
       9077146., 9077931., 9086657., 9056971., 9083352., 9066076.,
       9081899., 9064716., 9089857., 9065881., 9076491., 9070535.,
       9060360., 9068005., 9076700., 9088704., 9079557., 9080827.,
       9078499., 9045771., 9100555., 9062434., 9073295., 90915

In [10]:
sa

(True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 Fa